In [1]:
#%pip uninstall torch torchvision torchaudio -y

In [2]:
#%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu130

In [3]:
!nvidia-smi

Tue Nov  4 17:23:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 581.57                 Driver Version: 581.57         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   42C    P8              6W /   42W |     110MiB /   8151MiB |     28%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import torch

In [5]:
print(torch.backends.cudnn.enabled)

True


In [6]:
print(torch.version.cuda)

13.0


In [7]:
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

PyTorch version: 2.9.0+cu130
CUDA available: True
CUDA version: 13.0
Device name: NVIDIA GeForce RTX 5050 Laptop GPU


In [8]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import os


In [9]:
pwd

'c:\\Users\\muska\\OneDrive\\Desktop\\Unified_metor_internship\\Projects\\animal_classification\\Animal Classification'

In [10]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Verify it was set (optional)
print(f"CUDA_LAUNCH_BLOCKING is set to: {os.environ.get('CUDA_LAUNCH_BLOCKING')}")

CUDA_LAUNCH_BLOCKING is set to: 1


In [11]:
data_dir = "dataset"  # path to your dataset
num_classes = 15
batch_size = 32
num_epochs = 10
learning_rate = 1e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [12]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [13]:
full_dataset = datasets.ImageFolder(data_dir, transform=train_transform)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])
print(len(train_dataset))
print(len(val_dataset))

1555
389


In [14]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [15]:
model = models.resnet50(pretrained=True)

C:\Users\muska\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\muska\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)

model = model.to(device)

# =====================
# Loss & Optimizer
# =====================
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.fc.parameters(), lr=learning_rate)

In [17]:
for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    train_acc = 100 * correct / total
    train_loss = running_loss / total
    
    # Validation
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()
    
    val_acc = 100 * val_correct / val_total
    val_loss = val_loss / val_total
    
    print(f"Epoch [{epoch+1}/{num_epochs}] | "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

Epoch [1/10] | Train Loss: 2.4879 | Train Acc: 29.07% | Val Loss: 2.2218 | Val Acc: 54.76%
Epoch [2/10] | Train Loss: 2.0219 | Train Acc: 69.90% | Val Loss: 1.8055 | Val Acc: 82.78%
Epoch [3/10] | Train Loss: 1.6412 | Train Acc: 85.72% | Val Loss: 1.4705 | Val Acc: 87.66%
Epoch [4/10] | Train Loss: 1.3626 | Train Acc: 89.07% | Val Loss: 1.2266 | Val Acc: 91.00%
Epoch [5/10] | Train Loss: 1.1375 | Train Acc: 90.42% | Val Loss: 1.0457 | Val Acc: 89.20%
Epoch [6/10] | Train Loss: 0.9584 | Train Acc: 92.99% | Val Loss: 0.9149 | Val Acc: 91.26%
Epoch [7/10] | Train Loss: 0.8378 | Train Acc: 93.12% | Val Loss: 0.7671 | Val Acc: 93.06%
Epoch [8/10] | Train Loss: 0.7440 | Train Acc: 94.34% | Val Loss: 0.7213 | Val Acc: 92.03%
Epoch [9/10] | Train Loss: 0.6714 | Train Acc: 94.15% | Val Loss: 0.6584 | Val Acc: 91.77%
Epoch [10/10] | Train Loss: 0.6055 | Train Acc: 94.66% | Val Loss: 0.5649 | Val Acc: 93.06%
